This file will contain our experiments and a guide for understanding the output. The full report is available here: INSERT LINK TO REPORT

Let's get started! First, we'll import whatever we find necessary.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

print("Done importing")

Done importing


Recall the linear optimization algorithms described in section SECTION NUMBER. Below is an implementation of Algorithm ALGORITHM_NUMBER_FROM_REPORT.

In [0]:
# TODO @ Prabhat: Implement
def linear_optimization():
  pass

Now, let's generalize this to convex functions. In order to do this, recall that we need a gradient estimator. The next few cells contain our implementation of a spherical gradient estimator. 

First, we'll have to sample from a sphere and a ball.

In [0]:
def sample_from_sphere(d):
  p = np.zeros(d)
  theta = np.random.uniform(0, 2 * np.pi, d)
  p[0] = np.cos(theta[0])
  for i in range(1, d - 1):
    p[i] = p[i - 1] * np.cos(theta[i]) * np.sin(theta[i - 1]) / np.cos(theta[i - 1])
  p[d - 1] = 1
  for theta_i in theta[:-1]:
    p[d - 1] *= np.sin(theta_i)
  return p

def sample_from_ball(d):
  r = np.random.uniform(0, 1, 1)
  p = sample_from_sphere(d)
  return r * p



To check that our functions return valid samples, we can run the below cell. Observe that if valid samples are returned, then they must have the correct distribution as argued in SECTION_NUMBER of the report. 

In [21]:
def check_sampler():
  # these are parameters for the experiment. Change them to whatever.
  # note that we only need to check the spherical sampler
  d = 20
  runs = 100
  
  samples = [sample_from_sphere(d) for i in range(runs)]
  
  # check that all the samples lie on the sphere
  counter = 0
  for sample in samples:
    r = 0
    for coordinate in sample:
      r += np.square(coordinate)
    if r <= 0.99 or r >= 1.01:
      print("invalid sample")
      print(sample)
      print(counter)
      return
    counter += 1
  
check_sampler()
print("Done checking sampler")

Done checking sampler


We're now ready to implement our online gradient descent algorithm. Here, we begin our experimentation.

As a preliminary experiment, we consider the following problem setting. Suppose $x \in \mathbb{R}^d$ is $k$-sparse - that is, let's say that there are exactly $k$ nonzero elements in $x$. Now, let's say that we have $m < d$ measurements of $A_i^Tx + b_i$ where $b$ is small Gaussian noise. Our goal is to recover $x$. 

Recall that under certain conditions, we can nearly recover $x$ using the following convex relaxation of our problem given some regularization parameter $\lambda$:
\begin{align*}
  \min &\ \frac{1}{2}\left\lVert Ax - b\right\rVert_2^2 + \lambda\left\lVert x\right\rVert_1
\end{align*}
It's conceivable that $m$ is sufficiently large that we can't load all of $A$ into memory at once. Instead, we might load some subsets of the observations and $A_i$s into memory at each step in our optimization process. Furthermore, suppose that we don't have access to a gradient oracle; we only have access to the actual loss values instead.

Observe that this problem can now be solved using our BCO model. The set of losses that the (oblivious) adversary can select is the union of all $ 